In [1]:
import pandas as pd

import library.IO as myio
import library.motion as mt
import library.comparison as cp
from library.constants import *
import matplotlib

matplotlib.use('Qt5Agg')

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [5]:
file_name_0 = "P01_B_cut"
marks_motion_0 = [7600, 7850]
label_motion_0 = "label_0"

file_name_1 = "P02_B_cut"
#marks_motion_1 = [7600, 8100]
marks_motion_1 = [100, 600]
label_motion_1 = "label_1"

weights_0 = {}
for joint in SIMPLIFIED_JOINTS:
    weights_0[joint] = 1
weights_0["Right Hand"] = 0
weights_0["Right Upper Arm"] = 0
weights_0["Right Forearm"] = 0

# if you need more than one group of weight, just use the code below and add it to weights_groups
# weights_1 = {}
# for joint in SIMPLIFIED_JOINTS:
#     weights_1[joint] = 1

weights_groups = pd.DataFrame([weights_0])

flag_output_xlsx = False  # output xlsx or not
output_types = ["Score", "Segment Position", "Segment Velocity", "Segment Acceleration", "Segment Angular Velocity", "Segment Angular Acceleration"]
xlsx_filename = "example"

flag_show_animation = True  # generate the animation and play it
flag_output_gif = True  # generate the animation and output as gif
gif_file_name = "example"
flag_visualized_vector = True  # animation with visualized velocity or not
flag_heatmap = True  # animation with heatmap or not
flag_repeat = True  # animation will always repeat or not
visualized_vector = "Segment Velocity"
heatmap_recording = "Score"

In [6]:
meta_data_0 = mt.MetaData(file_name_0, marks_motion_0[0], marks_motion_0[-1], label_motion_0)
meta_data_1 = mt.MetaData(file_name_1, marks_motion_1[0], marks_motion_1[-1], label_motion_1)

if flag_output_xlsx:
    xlsx_settings = myio.XlsxSetting(
        xlsx_filename=xlsx_filename,
        output_types=output_types
    )
else:
    xlsx_settings = None

if flag_output_gif or flag_show_animation:
    animation_settings = myio.AnimationSetting(
        flag_visualized_vector=flag_visualized_vector,
        flag_heatmap=flag_heatmap,
        flag_repeat=flag_repeat,
        visualized_vector=visualized_vector,
        heatmap_recording=heatmap_recording
    )
else:
    animation_settings = None

example_io = myio.MyIO(
    flag_output_xlsx=flag_output_xlsx,
    flag_show_animation=flag_show_animation,
    flag_output_gif=flag_output_gif,
    xlsx_settings=xlsx_settings,
    animation_settings=animation_settings,
    motions_meta=[meta_data_0, meta_data_1]
)

In [7]:
motions = example_io.get_motions()
comparison = cp.Comparison(weights_groups, marks_motion_0)

generating label_0 from P01_B_cut
generating label_1 from P02_B_cut


In [8]:
for motion in motions:
    motion.centre().confront()
motions[1].synchronized_by(motions[0])

# import copy
#
# motion_1 = copy.deepcopy(motions[0])
# motions[0].confront()


placing label_0 in the centre
rotating label_0 to positive direction
placing label_1 in the centre
rotating label_1 to positive direction
synchronizing label_1 by label_0


In [210]:
import library.animation as animation

motions[0].meta.label = "Reference"
motions[1].meta.label = "Challenger"
ani = animation.Animation([motions[0],motions[1]], animation_settings)
ani.to_gif()
ani.show()

MovieWriter imagemagick unavailable; using Pillow instead.


generating the animation
saving animation as gif


In [9]:
result = comparison.compare(motions[0], motions[1], example_io.get_comparison_types())

calculating the distances and score


In [11]:
example_io.python_application_output(motions,result)

generating the animation


MovieWriter imagemagick unavailable; using Pillow instead.


saving animation as gif


KeyboardInterrupt: 

In [ ]:
import copy

motion_1 = copy.deepcopy(motions[0])
motions[0].confront()

In [169]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Line3D

fig = plt.figure()
axes = fig.add_subplot(1, 1, 1, projection="3d")
axes.set(xlim3d=(-1, 1), xlabel='X')
axes.set(ylim3d=(-1, 1), ylabel='Y')
axes.set(zlim3d=(-1, 1), zlabel='Z')

for index, joint_connection in enumerate(SKELETON_CONNECTION_MAP):
    endpoint_0: str = joint_connection[0]
    endpoint_1: str = joint_connection[1]

    line_position = []
    position_0 = motion_1.recordings["Segment Position"]
    for axis in AXIS:
        line_position.append([position_0[endpoint_0 + axis].iloc[-1], position_0[endpoint_1 + axis].iloc[-1]])
    line = Line3D(line_position[0],line_position[1], line_position[2], color='red')
    if index == 0:
        line.set_label("Before Confronting")
    axes.add_line(line)

    line_position = []
    position = motions[0].recordings["Segment Position"]
    for axis in AXIS:
        line_position.append([position[endpoint_0 + axis].iloc[-1], position[endpoint_1 + axis].iloc[-1]])

    line = Line3D(line_position[0],line_position[1], line_position[2], color='darkred')
    if index == 0:
        line.set_label("After Confronting")
    axes.add_line(line)

axes.legend()

plt.show()
fig.savefig('temp.png', transparent=True)

In [187]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from typing import List
fig = plt.figure()
axes = fig.add_subplot(1, 1, 1, projection="3d")
axes.set(xlim3d=(-1, 1), xlabel='X')
axes.set(ylim3d=(-1, 1), ylabel='Y')
axes.set(zlim3d=(-1, 1), zlabel='Z')

points = motions[0].recordings["Segment Position"][["Right Hand x", "Right Hand y", "Right Hand z", "Frame"]].values
index = motions[0].recordings["Segment Position"][["Frame"]].astype(int).values
norm = plt.Normalize(index.min(), index.max())
norm_y = norm(index)[:,0]
axes.scatter(points[:,0], points[:,1], points[:,2], c=list(norm_y), cmap="Reds", s=2)

points = motions[1].recordings["Segment Position"][["Right Hand x", "Right Hand y", "Right Hand z", "Frame"]].values
index = motions[1].recordings["Segment Position"][["Frame"]].astype(int).values
norm = plt.Normalize(index.min(), index.max())
norm_y = norm(index)[:,0]
axes.scatter(points[:,0], points[:,1], points[:,2], c=list(norm_y), cmap="Greens", s=2)
axes.view_init(20, 10, "z")
axes.legend(handles=[mpatches.Patch(color='darkred', label='Reference'), mpatches.Patch(color='green', label='Challenger')])
for index, joint_connection in enumerate(SKELETON_CONNECTION_MAP):
    endpoint_0: str = joint_connection[0]
    endpoint_1: str = joint_connection[1]

    line_position = []
    position_0 = motions[0].recordings["Segment Position"]
    for axis in AXIS:
        line_position.append([position_0[endpoint_0 + axis].iloc[-2], position_0[endpoint_1 + axis].iloc[-1]])
    line = Line3D(line_position[0],line_position[1], line_position[2], color='darkred')
    if index == 0:
        line.set_label("Reference")
    axes.add_line(line)

    line_position = []
    position = motions[1].recordings["Segment Position"]
    for axis in AXIS:
        line_position.append([position[endpoint_0 + axis].iloc[-2], position[endpoint_1 + axis].iloc[-1]])

    line = Line3D(line_position[0],line_position[1], line_position[2], color='green')
    if index == 0:
        line.set_label("Challenger")
    axes.add_line(line)


plt.show()
fig.savefig('temp.png', transparent=True)